In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/kaggle/input/vehicle-dataset-from-cardekho/car data.csv')

In [ ]:
df

In [ ]:
df.info()

In [ ]:
sns.boxplot(data=df,y='Selling_Price',x='Fuel_Type')

In [ ]:
sns.boxplot(data=df,y='Selling_Price',x='Seller_Type')

In [ ]:
sns.scatterplot(data=df,x='Present_Price',y='Selling_Price')

In [ ]:
car_df = pd.get_dummies(df.drop(['Car_Name','Year'],axis=1),drop_first=True)

In [ ]:
car_df

In [ ]:
sns.heatmap(car_df.corr(),annot=True)

In [ ]:
X = car_df.drop('Selling_Price',axis=1)

In [ ]:
y = car_df['Selling_Price']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
X_train = X_train.T
X_test = X_test.T

In [ ]:
y_train = y_train.values.reshape(1,-1)
y_test = y_test.values.reshape(1,-1)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
X_train[:,[2,4,8,12]]

In [ ]:
def initialize_parameters(layer_dims):
    L= len(layer_dims)
    W1 = np.random.randn(layer_dims[1],layer_dims[0])*np.sqrt(2/layer_dims[0])
    b1 = np.zeros((layer_dims[1],1))
    W2 = np.random.randn(layer_dims[2],layer_dims[1])*np.sqrt(2/layer_dims[1])
    b2 = np.zeros((layer_dims[2],1))
    W3 = np.random.randn(layer_dims[3],layer_dims[2])*np.sqrt(2/layer_dims[2])
    b3 = np.zeros((layer_dims[3],1))
    
    parameters = W1,b1,W2,b2,W3,b3
    return parameters
    

In [ ]:
def relu(x):
    return np.maximum(0,x)

In [ ]:
def propagation(X,y,parameters):
    m = X_train.shape[1]
    W1,b1,W2,b2,W3,b3 = parameters
    
    Z1 = np.dot(W1,X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2,A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3,A2) + b3
    A3 = relu(Z3)
    
    dA3 = A3 - y
    dZ3 = np.multiply(dA3, np.int64(A3 > 0))
    dW3 = (1/m)*np.dot(dZ3,A2.T)
    db3 = (1/m)*np.sum(dZ3,axis=1,keepdims=True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1./m * np.dot(dZ2, A1.T)
    db2 = 1./m * np.sum(dZ2, axis=1, keepdims = True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1./m * np.dot(dZ1, X.T)
    db1 = 1./m * np.sum(dZ1, axis=1, keepdims = True)
    
    grads = dW1,db1,dW2,db2,dW3,db3
    return A3,grads

In [ ]:
def optimize(parameters,grads,learning_rate):
    W1,b1,W2,b2,W3,b3 = parameters
    dW1,db1,dW2,db2,dW3,db3 = grads
    
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2
    W3 = W3 - learning_rate*dW3
    b3 = b3 - learning_rate*db3
    
    parameters = W1,b1,W2,b2,W3,b3
    return parameters

    

In [ ]:
def gradient_descent(X,y,learning_rate=0.01,num_iterations=500,layer_dims=[X_train.shape[0],10,10,1]):
    parameters = initialize_parameters(layer_dims)
    for i in range(num_iterations):
        A3,grads = propagation(X,y,parameters)
        parameters = optimize(parameters,grads,learning_rate)
        
    return parameters
        

In [ ]:
parameters = gradient_descent(X_train,y_train,learning_rate=0.01,num_iterations=500,layer_dims=[X_train.shape[0],12,10,1])

In [ ]:
pred,_ = propagation(X_test,y_test,parameters)

In [ ]:
y_test.shape

In [ ]:
pred.shape

In [ ]:
sns.scatterplot(y_test[0],pred[0])

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [ ]:
MAE = mean_absolute_error(y_test[0],pred[0])

In [ ]:
MAE

In [ ]:
RMSE = np.sqrt(mean_squared_error(y_test[0],pred[0]))

In [ ]:
RMSE

In [ ]:
R2 = r2_score(y_test[0],pred[0])

In [ ]:
R2